In [19]:
from amadeus import Client, ResponseError
import requests
import json
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [44]:
# Your API credentials
client_id = '2MDgHHj2hwOhMgLOY2KKfKsSwoqkLd6Z'
client_secret = 'ZFxpu7xyjRmGUKCq'

# Requesting the access token
url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    access_token = response.json().get('access_token')
    print("Access Token:", access_token)
else:
    print("Failed to get access token:", response.text)


Access Token: hnaGexax74G9gqlvZV58Y6LzmkrQ


In [46]:
data_path = "C:/Users/alexa/Python Projects/OfferFligthApp/airports.dat"
column_names = ["id", "name", "city", "country", "iata_code", "icao_code", "latitude",
               "longitude", "altitude", "timezone_offset", "DST", "timezone_name", "type",
               "source"]
df = pd.read_csv(data_path, names = column_names,delimiter = ',')
print(df.head(5))

   id                                         name          city  \
0   1                               Goroka Airport        Goroka   
1   2                               Madang Airport        Madang   
2   3                 Mount Hagen Kagamuga Airport   Mount Hagen   
3   4                               Nadzab Airport        Nadzab   
4   5  Port Moresby Jacksons International Airport  Port Moresby   

            country iata_code icao_code  latitude   longitude  altitude  \
0  Papua New Guinea       GKA      AYGA -6.081690  145.391998      5282   
1  Papua New Guinea       MAG      AYMD -5.207080  145.789001        20   
2  Papua New Guinea       HGU      AYMH -5.826790  144.296005      5388   
3  Papua New Guinea       LAE      AYNZ -6.569803  146.725977       239   
4  Papua New Guinea       POM      AYPY -9.443380  147.220001       146   

  timezone_offset DST         timezone_name     type       source  
0              10   U  Pacific/Port_Moresby  airport  OurAirports  
1   

In [47]:
df_filtered = df[['city', 'iata_code', 'name']].dropna()
print(df_filtered.head())

           city iata_code                                         name
0        Goroka       GKA                               Goroka Airport
1        Madang       MAG                               Madang Airport
2   Mount Hagen       HGU                 Mount Hagen Kagamuga Airport
3        Nadzab       LAE                               Nadzab Airport
4  Port Moresby       POM  Port Moresby Jacksons International Airport


In [48]:
def get_iata_codes_by_city(city_name):
    codes = df_filtered[df_filtered['city'].str.lower() == city_name.lower()]['iata_code'].dropna().unique()
    return codes if len(codes) > 0 else ["IATA code not found"]
def get_iata_codes_by_name(airport_name):
    codes = df_filtered[df_filtered['name'].str.lower() == airport_name.lower()]['iata_code'].dropna().unique()
    return codes if len(codes) > 0 else ["IATA code not found"]
    
city_name = input("Enter the name of the city: ")
iata_code = get_iata_codes_by_city(city_name)
print(f"The IATA code for {city_name} is: {iata_code}")

airport_name = input("Enter the name of the airport: ")
iata_code = get_iata_codes_by_name(airport_name)
print(f"The IATA code for {airport_name} is: {iata_code}")

Enter the name of the city:  


The IATA code for  is: ['IATA code not found']


Enter the name of the airport:  


The IATA code for  is: ['IATA code not found']


In [24]:
# Function to plot airports for a given city
def plot_airports_for_city(city_name):
    # Filter data for the specified city
    city_airports = df[df['city'].str.lower() == city_name.lower()]

    # Check if any airports were found
    if city_airports.empty:
        print(f"No airports found for the city: {city_name}")
        return

    # Create a Folium map centered around the first airport in the city
    first_airport = city_airports.iloc[0]
    city_map = folium.Map(location=[first_airport['latitude'], first_airport['longitude']], zoom_start=10)

    # Add a marker cluster to group airport markers
    marker_cluster = MarkerCluster().add_to(city_map)

    # Add each airport in the city to the map
    for _, airport in city_airports.iterrows():
        folium.Marker(
            location=[airport['latitude'], airport['longitude']],
            popup=f"{airport['name']} ({airport['iata_code']})",
            tooltip=airport['name']
        ).add_to(marker_cluster)

    # Display the map
    return city_map

In [49]:
iata_to_name = pd.Series(df_filtered.name.values, index=df_filtered.iata_code).to_dict()

def get_airport_name_by_iata(iata_code):
    return iata_to_name.get(iata_code.upper(), "IATA code not found")

In [60]:
#In this section, we ask the user where is he located, then we fetch all the possible
#iata codes from the city, and we call the API to display offer fligths that have of 
#origin such city. 
city_or_airport = input("Select current city or airport-name: ")
if(city_or_airport == "city"): 
    city_name = input("From what city do you wish to explore possible offers: ")
    iata_list =  get_iata_codes_by_city(city_name)
    print(iata_list)
    for iata in iata_list: 
        airport_name = get_airport_name_by_iata(iata)
        choice = input(f"Do you wish to flight from {iata}: {airport_name}")
        if(str.lower(choice) == "yes"): 
            origin_city = iata
            break
        else: 
            continue
    max_price = input("What is the max amount for offer price (in dollars): ")
else: 
    airport_name = input("What is the name of the airport: ")
    iata_code = get_iata_codes_by_name(airport_name)
    print(iata_code)
    origin_city = iata_code
    max_price = input("What is the max amount for offer price (in dollars): ")

Select current city or airport-name:  city
From what city do you wish to explore possible offers:  Mexico


['IATA code not found']


Do you wish to flight from IATA code not found: IATA code not found no
What is the max amount for offer price (in dollars):  no


In [58]:
# Using the access token to call the Flight Inspiration Search API
destination_url = 'https://test.api.amadeus.com/v1/shopping/flight-destinations'
headers = {
    'Authorization': f'Bearer {access_token}'
}
params = {
    'origin': origin_city,  
    #'destination': destination_city,
    'maxPrice': max_price   
}

get_server_information = requests.get(destination_url, headers=headers, params=params)

if response.status_code == 200:
    #print("Available Destinations:", response.json())
    data = get_server_information.json()
    pretty_data = json.dumps(data, indent = 4)
    print(pretty_data)
else:
    print("Failed to retrieve destinations:", response.text)

{
    "errors": [
        {
            "status": 404,
            "code": 6003,
            "title": "ITEM/DATA NOT FOUND OR DATA NOT EXISTING",
            "detail": "No price result found"
        }
    ]
}


In [55]:
destinations = []
prices = []
departure_dates = []
return_dates = []

# Loop through each flight destination in the JSON response
for flight in data["data"]:
    # Append each attribute to its corresponding list
    destinations.append(flight["destination"])
    prices.append(float(flight["price"]["total"]))  # Convert to float for easier manipulation if needed
    departure_dates.append(flight["departureDate"])
    return_dates.append(flight["returnDate"])
i = 0
for offer in destinations:
    name = get_airport_name_by_iata(offer)
    print(f"You can flight to {offer} (airport name: {name}).\nFor the price of: {prices[i]}. \nDeparture date: {departure_dates[i]}. \nReturn date: {return_dates[i]}.")
    print("------------------------------------------------------------------------")
    ++i

You can flight to LGA (airport name: La Guardia Airport).
For the price of: 72.95. 
Departure date: 2024-12-18. 
Return date: 2024-12-31.
------------------------------------------------------------------------


In [ ]:
# Input city name and display the map
city_name = input("Enter the name of the city: ")
city_map = plot_airports_for_city(city_name)
if city_map:
    city_map.save("airports_map.html")  # Saves map to an HTML file
    display(city_map)  # This will show the map if you are in a Jupyter Notebook